In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install fasttext
!pip install pyvi

     |████████████████████████████████| 61kB 1.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2391080 sha256=8c364cc082f4355d68e971f7661c7b1d2962065cec10ed6402f6d4f9443a4d8c
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
     |████████████████████████████████| 8.8MB 44.1MB/s 
     |████████████████████████████████| 757kB 63.4MB/s 


In [0]:
import pandas as pd
import numpy as np
import re
import fasttext
# from pyvi import ViTokenizer
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [0]:
ROOT_PATH = '/content/drive/My Drive/Colab Notebooks (1)/sentiment_analysis'

## Load and process data

In [20]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks (1)/sentiment_analysis/data/train_add_gen.csv', encoding='utf-16')
print(train.shape)
# train['content][:20]

(10187, 2)


In [8]:
from collections import Counter
Counter(train.ground)

Counter({0: 5084, 1: 5103})

In [10]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks (1)/sentiment_analysis/data/test.csv', encoding='utf-16')
print(test.shape)
test.head()

(1552, 2)


,content,ground
0,Sản phẩm hay bị đơ! Dùng pin 4g hao nhanh. Hỗ ...,0
1,"Đẹp nhất, cấu hình cao nhất, pin trâu nhất tro...",1
2,"Trên cả tuyệt vời 😍 pin trâu, mượt, sang trọng...",1
3,Sản phẩm tầm trung Xài tạm ổn. Nói chung sản p...,0
4,oppo neo 7 thật quá đã. Tính năng cũng như HĐH...,1


In [0]:
def text_normalize(df, tokenize=False):
    # Convert text to lowercase
    df['content'] = df['content'].str.lower()
    # Remove numbers and words with numbers
    df['content'] = df['content'].str.replace('\w*\d\w*', ' ')
    # Remove punctuation
    df['content'] = df['content'].str.replace('[^\w\s]', ' ')
    # Remove whitespaces
    df['content'] =  df['content'].str.split().apply(lambda x : ' '.join(word for word in x))
    # Tokenize
    if tokenize:
        df['content'] = df['content'].apply(lambda x : ViTokenizer.tokenize(x))

In [12]:
text_normalize(train)
text_normalize(test)
test.tail()

,content,ground
1547,máy mình dùng một thời gian ngắn nhưng chậm sử...,0
1548,cho đến hôm nay máy vẫn bị loạn cảm ứng cập nh...,0
1549,đẹp chuẩn sài rất êm tôi rất hài lòng về dòng ...,1
1550,mặc dù mua máy cũ nhưng trông như máy mới mọi ...,1
1551,máy chạy tốt mình hài lòng mua từ tháng năm tớ...,0


In [0]:
texts = train['content'].tolist()
labels = train['ground'].tolist()

## FastText Estimator Class

In [0]:
# Convert file to FastText format
def store_file_fasttext_format(outfile_dir, texts, labels):
    """
    Store texts and labels to fasttext format file
    """
    with open(outfile_dir, 'w') as f:
        for i in range(0, len(texts)):
            f.write('__label__%s %s\n' % (labels[i], texts[i]))

In [0]:
class FastTextEstimator(BaseEstimator):
    def __init__(self, model_dir, epoch_val=5, lr_val=0.1):
        """Initial model and model directory"""
        self.model_dir = model_dir
        self.model = None
        self.epoch_val = epoch_val
        self.lr_val = lr_val
    def fit(self, texts, labels):
        """Fit the model using `texts` and `labels` in fasttext format

        Parameters
        ----------
        texts : list
        labels : list
        """
        # Save texts and labels in fasttext format file
        store_file_fasttext_format(
            self.model_dir+'/content/train.txt',
            texts,
            labels)
        # Train with specific hyperparameters
        self.model = fasttext.train_supervised(
            self.model_dir+'/content/train.txt',
            dim=300,
            wordNgrams=5,
            # pretrainedVectors=self.model_dir+'cc.vi.300.vec',
            epoch=self.epoch_val,
            verbose=1,
            lr=self.lr_val)
        return self
    def predict(self, texts):
        """Predict the class labels for the provided texts

        Parameters
        ----------
        texts : list

        Returns: class labels for each text
        """
        predicted_labels = []
        predictions = self.model.predict(texts)[0]
        # Convert predicted labels from text to string for f1 score function
        for prediction in predictions:
            predicted_labels.append(0 if prediction[0] == '__label__0' else 1)
        return np.array(predicted_labels)
    def score(self, texts, labels):
        """Compute accuracy score for the given text and prediction

        Parameters
        ----------
        texts : list
        labels : list
        
        Return: accuracy score
        """
        predicted_labels = self.predict(texts)
        return [accuracy_score(labels, predicted_labels),
                f1_score(labels, predicted_labels)]

## Train, test

In [21]:
MODEL_DIR = '/content/'
clf = FastTextEstimator(model_dir=MODEL_DIR)
clf.__init__(model_dir=ROOT_PATH, epoch_val=10, lr_val=0.1)
scores = cross_val_score(
    clf,
    texts,
    labels,
    cv=StratifiedKFold(n_splits=10),
    scoring='accuracy')
print(scores)
scores.mean()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


FileNotFoundError: ignored

In [27]:
%%time
clf = FastTextEstimator(model_dir='')
clf.__init__(model_dir='', epoch_val=29, lr_val=0.1)
clf.fit(texts, labels)

CPU times: user 44.2 s, sys: 1 s, total: 45.2 s
Wall time: 45.3 s


In [0]:
clf.score(test['content'].tolist(), test['ground'].tolist())

[0.9039948453608248, 0.908420405654579]

In [0]:
y_pred = clf.predict(test['content'].tolist())

In [0]:
print('Ground\tPred\tText')
L1 =[]
L2 =[]
L3 = []
for i in range(len(test)):
    if test['ground'].tolist()[i] != y_pred[i]:
        print(str(test['ground'].tolist()[i]) + '\t' + str(y_pred[i]) + '\t' + test['content'][i])
        L1.append(str(test['ground'].tolist()[i]))
        L2.append(str(y_pred[i]))
        L3.append(test['content'][i])

df = pd.DataFrame({'Label': L1, 'Pred': L2,'Text': L3})
df.to_csv(path_or_buf = '/content/drive/My Drive/sentiment_analysis/label_predict.csv',index=False)

Ground	Pred	Text
0	1	sản phẩm tầm trung xài tạm ổn nói chung sản phâm bình thường không có ưu điểm nổi bât
1	0	oppo neo thật quá đã tính năng cũng như hđh phù hợp với giới trẻ hiện nay nhưng giá cả nê giảm lại một chút nữa thì bán chạy hơn nhiều
0	1	sản phẩm tạm ổn cấu hình yếu quá nokia chạy hệ điều hành adroi chưa thực sự tốt
0	1	dùng ok hiện tại thì chưa thấy có vấn đề gì cả chỉ có cái dung lượng pin hơi ít chơi game mượt chụp buổi tối thì cam hơi kém chụp đủ sáng thì ok
1	0	rất ok nhưng mau hết pin quá m k chơi game gì cả mà một ngày vẫn sạc
0	1	máy đẹp chụp hình khỏi chê nhưng chụp hình hay chơi game tầm thì máy hơi nóng
1	0	đã cập nhật phiên bản mới từ samsung sau bao lần vất vả cảm nhận hầu hết fix các lỗi mà các bác đã nói trên đây chưa thử liên quân ai mà cập nhật không được hay bị lỗi cảm ứng v v thì mình khuyến cáo nên đem ra trung tâm bảo hành gần nhất của samsung để họ cài lại phần mềm phần mềm đây là phần mềm gốc chỉ ở trung tâm mới có chứ không phải là khôi phục cài đặt 

In [0]:
savefile = ROOT_PATH + 'resource/fasttext_0102.bin'

In [0]:
# save model to disk
model = clf.model
model.save_model(savefile)

In [0]:
# load model
ft_model = fasttext.load_model(savefile)

ft_model.predict(test['content'].tolist())

([['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__label__1'],
  ['__label__1'],
  ['__label__0'],
  ['__label__0'],
  ['__label__1'],
  ['__label__0'],
  ['__labe